# Introduction

### Import the libraries

In [1]:
import numpy as np
import pandas as pd
import matplotlib as mpl
import matplotlib.pyplot as plt

ModuleNotFoundError: No module named 'pandas'

### Notebook display settings

In [ ]:
pd.set_option('display.max_rows', 100)
pd.set_option('display.max_columns', 50)

### Default figure settings

In [ ]:
mpl.rcParams['axes.titlesize'] = 18
mpl.rcParams['axes.labelsize'] = 16
mpl.rcParams['axes.titleweight']='bold'
mpl.rcParams['axes.labelweight']='normal'
mpl.rcParams['figure.figsize']=(8,6)
mpl.rcParams['font.family'] = "sans-serif"
mpl.rcParams['font.sans-serif'] = "Arial"

# Data Transformation & Exploration

### Read the data

In [2]:
training_data=pd.read_csv('Data/epl-training.csv')
# Dropping the trailing columns
training_data.dropna(axis=1, how='all',inplace=True)
training_data.drop([column for column in training_data.columns if 'Unnamed' in column], axis = 1, inplace=True) 
training_data.shape

NameError: name 'pd' is not defined

In [3]:
training_data.head()

NameError: name 'training_data' is not defined

### Exploratory Data Analysis

In [4]:
def get_pivot_column_vs_column_count(df,col1,col2):
    return df.groupby([col1,col2]).size().reset_index().rename(columns={0:'counts'}).\
              pivot(values='counts',index=col1,columns=col2)

In [5]:
def get_column_countplot(df,col,y='FTR',ax=None ):
    if ax is None:
        ax = plt.gca()
    df.groupby([col, y]).size().reset_index().\
       pivot(columns=y, index=col, values=0).\
       plot(kind='bar', stacked=True,ax=ax)
    ax.set_xlabel(col)
    ax.set_ylabel('Counts')
    ax.set_title('Count Plot: %s'%(col))

#### Number of matches

In [6]:
get_pivot_column_vs_column_count(training_data,'HomeTeam','AwayTeam')

NameError: name 'training_data' is not defined

### HTR vs. FTR

In [7]:
get_pivot_column_vs_column_count(training_data,'HTR','FTR')

NameError: name 'training_data' is not defined

In [8]:
cat_col_list=[x for x in training_data.columns if x not in ['Date','HomeTeam','AwayTeam','Referee','FTR','HTR']]

NameError: name 'training_data' is not defined

In [9]:
cat_col_list = np.reshape(cat_col_list, (8, 2))

NameError: name 'cat_col_list' is not defined

In [10]:
fig, axes = plt.subplots(8,2,figsize=(15,60))
for i in range(8):
    for j in range(2):
        get_column_countplot(training_data,cat_col_list[i][j], ax=axes[i,j])
fig.tight_layout(rect=[0, 0.03, 1, 0.97])
fig.suptitle("Count plots of all features",fontweight='bold',fontsize=20)
plt.show()

NameError: name 'plt' is not defined

# Methodology Overview

# Model Training & Validation

# Results

# Final Predictions on Test Set

# Conclusion